In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector

#from custom_packages.email_operations import EmailOperations

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

pd.set_option('display.float_format', lambda x: '%.2f' % x)



In [0]:
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')



print(today)
print(yesterday)


last_2_wks = datetime.today() - timedelta(days = 14)
last_2_wks = last_2_wks.strftime('%Y-%m-%d')
print('------------------------------------')
print(last_2_wks)

print('\n')
now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M:%S')
trunc_last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M')
print(last_30_mins, 'to', now)

In [0]:
username = data['auto-report']['email']
passwd = data['auto-report']['passwd']
print(username)



SUSPICIOUS TRANSACTIONS ALERT

 

Large/frequent cash deposit into customer’s account.
i.                     Tier 1: Single deposit transaction more that N100,000

ii.                   Tier 2: Single deposit of transaction more than N200,000

iii.                 Tier 3: Multiple of same value

In [0]:
%%time

sql = f"""
select
dat.client_id,
dat.client_name,
dat.account_no,
dat.product_name,
dac.client_tier,
transaction_type,
amount,
running_balance as balance,
transaction_date,
transaction_remarks,
email_address,
mobile_number
from dwh_all_transactions dat
left join dwh_all_clients dac on dac.client_id = dat.client_id
left join dwh_all_accounts daa on daa.account_no = dat.account_no

where dac.client_tier = 'Tier 1'
and left(dat.transaction_date, 10) = '{yesterday}'
and transaction_type_enum in (1, 2)
and is_reversed = 'No'
and amount >= 100000
and product_id in (29, 33) -- (26, 36, 24, 31, 32, 30)

and (transaction_remarks not like '%%FD Book%%' and transaction_remarks not like '%%Roll over%%' 
and transaction_remarks not like '%%Loan Disbu%%' and transaction_remarks not like '%%Partial Liquidat%%');
"""
tier1 = pd.read_sql_query(sql, conn)
tier1.head()

In [0]:
display(tier1)

In [0]:
%%time
sql = f"""
select
dat.client_id,
dat.client_name,
dat.account_no,
dat.product_name,
dac.client_tier,
transaction_type,
amount,
running_balance as balance,
transaction_date,
transaction_remarks,
email_address,
mobile_number
from dwh_all_transactions dat
left join dwh_all_clients dac on dac.client_id = dat.client_id
left join dwh_all_accounts daa on daa.account_no = dat.account_no
where dac.client_tier = 'Tier 2'
and left(dat.transaction_date, 10) = '{yesterday}'
and transaction_type_enum in (1, 2)
and is_reversed = 'No'
and amount >= 200000
and product_id in (29, 33)
and (transaction_remarks not like '%%FD Book%%' and transaction_remarks not like '%%Roll over%%' 
    and transaction_remarks not like '%%Loan Disbu%%' and transaction_remarks not like '%%Partial Liquid%%')
"""
tier2 = pd.read_sql_query(sql, conn)
tier2.head()

In [0]:
display(tier2)

In [0]:
%%time
sql = f"""
select
dat.client_id,
dat.client_name,
dat.account_no,
dat.product_name,
dac.client_tier,
transaction_type,
amount,
running_balance as balance,
transaction_date,
transaction_remarks,
email_address,
mobile_number
from dwh_all_transactions dat
left join dwh_all_clients dac on dac.client_id = dat.client_id
left join dwh_all_accounts daa on daa.account_no = dat.account_no
where dac.client_tier = 'Tier 3'
and left(dat.transaction_date, 10) = '{yesterday}'
and transaction_type_enum in (1, 2)
and is_reversed = 'No'
and amount >= 200000
and product_id in (33, 29) -- 26, 36, 24, 31, 32, 30)
and (transaction_remarks not like '%%FD Book%%' and transaction_remarks not like '%%Roll over%%' and transaction_remarks not like '%%Loan Disbu%%' 
and transaction_remarks not like '%%Partial Liquid%%')
order by 1
"""
tier3 = pd.read_sql_query(sql, conn)
tier3.head()

In [0]:
display(tier3)

In [0]:
tier3 = tier3[tier3['transaction_remarks'] != 'send Scarves OLUWATOBILOBA ODUBELA JOSEPH transfe/ To BADA SAMUEL OLATUNBOSUN']
tier3.head()

In [0]:
"""
SELECT
mc.id AS client_id,
mc.display_name AS client_name,
msa.account_no,
msp.name AS product_name,
case mc.client_level_cv_id 
	when '99' then 'Tier 1'
	when '100' then 'Tier 2'
	when '101' then 'Tier 3' END AS client_tier,
renum.enum_value as transaction_type,
st.amount,
st.running_balance_derived AS balance,
st.transaction_date,
ai.`Email Address` AS email_address,
ai.`Mobile Number` AS mobile_number
from m_savings_account_transaction st
left join m_savings_account msa on msa.id = st.savings_account_id
left join m_savings_product msp on msp.id = msa.product_id
left join m_client mc on mc.id = msa.client_id
left join m_group mg on mg.id = msa.group_id
left join (SELECT
       rv.enum_name,
       rv.enum_id,
       rv.enum_message_property,
       rv.enum_value
       FROM r_enum_value rv
       WHERE rv.enum_name = 'savings_transaction_type_enum') renum on renum.enum_id = st.transaction_type_enum
left join m_appuser map on map.id = st.appuser_id
left JOIN m_payment_detail mpd ON mpd.id = st.payment_detail_id
left JOIN m_payment_type mpt ON mpt.id = mpd.payment_type_id
left join m_transaction_request mtr on mtr.transaction_id = st.id
LEFT JOIN `Address Info` ai ON ai.client_id = mc.id
WHERE LEFT(st.transaction_date, 10) = '{yesterday}'
AND st.transaction_type_enum IN (1,2)
AND st.is_reversed_by_contra_entry = '0'
AND st.amount >= '200000'
AND msa.product_id IN (33,29)
AND (mtr.remarks  not like '%FD Book%' and mtr.remarks  not like '%Roll over%' and mtr.remarks  not like '%Loan Disbu%' 
and mtr.remarks not like '%Partial Liquid%')
"""

In [0]:
inf = duckdb.query(f"""
        select
        client_id,
        count(account_no),
        count(amount)
        from tier3
        where transaction_type = 'deposit'
        group by 1
        having count(account_no) > 1 and count(amount) > 1
        """).to_df()

inf3 = tier3[tier3['client_id'].isin(inf['client_id'].tolist())].reset_index(drop = True)

inf3.head()

In [0]:
out = duckdb.query(f"""
        select
        client_id,
        count(account_no),
        count(amount)
        from tier3
        where transaction_type = 'withdrawal'
        group by 1
        having count(account_no) > 1 and count(amount) > 1
        """).to_df()

out3 = tier3[tier3['client_id'].isin(out['client_id'].tolist())].reset_index(drop = True)

out3.head()

In [0]:
with pd.ExcelWriter(f"/Workspace/ReportDump/Tier_Level_Alert/Tier level {yesterday}.xlsx") as writer:
    tier1[tier1['transaction_type'] == 'deposit'].to_excel(writer, sheet_name = 'Tier 1 inflows', index = False)
    tier1[tier1['transaction_type'] == 'withdrawal'].to_excel(writer, sheet_name = 'Tier 1 outflows', index = False)
    tier2[tier2['transaction_type'] == 'deposit'].to_excel(writer, sheet_name = 'Tier 2 inflows', index = False)
    tier2[tier2['transaction_type'] == 'withdrawal'].to_excel(writer, sheet_name = 'Tier 2 outflows', index = False)
    inf3.to_excel(writer, sheet_name = 'Tier 3 inflows', index = False)
    out3.to_excel(writer, sheet_name = 'Tier 3 outflows', index = False)


In [0]:
# FILE TO SEND AND ITS PATH
filename = f'Tier level {yesterday}.xlsx'
SourcePathName  = "/Workspace/ReportDump/Tier_Level_Alert/" + filename


import csv, smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders



msg = MIMEMultipart()
msg['From'] = 'auto-report@vfdtech.ng'
msg['To'] = f'bolaji.jimoh@vfd-mfb.com, damilola.fatunmise@vfd-mfb.com, data-team@vfdtech.ng, joseph.ogbeche@vfd-mfb.com'
# msg['cc'] = 'productteam@vfd-mfb.com, salesteam@vfd-mfb.com'
msg['Bcc'] = ' data-team@vfdtech.ng'
msg['Subject'] = f'Tier Level Transaction Alert for {yesterday}'
body = f'Please find attached tier level transaction report as at {yesterday}'
msg.attach(MIMEText(body, 'plain'))


## ATTACHMENT PART OF THE CODE IS HERE
attachment = open(SourcePathName, 'rb')
part = MIMEBase('application', "octet-stream")
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(part)

server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.login(username, passwd)  ### if applicable
server.send_message(msg)
server.quit()

## Manually activated accounts 

 Please we want a daily auto generated report on all manually opened Corporate and Individual accounts. We will appreciate if this report can start coming in from tomorrow.

In [0]:
%%time
rmanual_accts = pd.read_sql(f'''
                select 
                distinct
                COALESCE(dac.client_id, dgc.group_client_id, daa.client_id) as client_id,
                dac.client_name,
                dac.client_tier,
                --dbw.bvn,
                dab.account_no,
                dbw.restriction_status,
                daa.product_id,
                daa.product_name,
                dab.balance,
                dab.approvedon_date as activation_date,
                dac.email_address,
                dac.mobile_number,
                dac.mailing_address,
                dab.request_sbumittedby_user_fullname as submission_request_username,
                dab.activatedby_user_fullname
                from dwh_account_balances dab
                left join dwh_all_accounts daa on daa.account_id = dab.account_id
                left join dwh_group_clients dgc on dgc.group_client_id = daa.client_id
                left join dwh_all_clients dac on dac.client_id = daa.client_id
                left join dwh_bvn_watchlist dbw on dbw.client_id = dac.client_id
                where dab.request_submittedby_userid  != 31
                and left(dab.approvedon_date, 10) = '{yesterday}' 
                and length(COALESCE(dac.client_id, dgc.group_client_id, daa.client_id)) > 1
                ''', conn)
rmanual_accts.head()

In [0]:
len(rmanual_accts)

In [0]:
## AML rating score 
aml = pd.read_sql(f'''
            select
            client_id as client_id,
            final_risk_score as aml_risk_score,
            case when final_risk_score <= 1.5 then 'Low risk'
                when final_risk_score > 1.5  and final_risk_score < 4 then 'Medium risk'
                when final_risk_score >= 4 then 'High risk' end as aml_risk_rating
            from aml_customer_risk_rating acr
            where client_id in ({','.join([str(x) for x in rmanual_accts['client_id'].tolist()])})
            ''', Dumpdb)

rmanual_accts = pd.merge(rmanual_accts, aml, on = 'client_id', how = 'left')

In [0]:
rmanual_accts.head()

In [0]:
officr = pd.read_sql(f'''
                select
                a.client_id,
                dac.client_id as account_officer_id,
                dac.client_name as account_officer_name
                from dwh_all_clients dac
                left join 
                            (select
                            client_id,
                            referred_by_id
                            from dwh_all_clients dcl
                            where client_id in ({','.join([str(x) for x in rmanual_accts['client_id'].tolist()])})) a on a.referred_by_id = dac.client_id
                where dac.client_id in (
                                    select
                                    referred_by_id
                                    from dwh_all_clients
                                    where client_id in ({','.join([str(x) for x in rmanual_accts['client_id'].tolist()])}))
                ''', conn)

rmanual_accts = pd.merge(rmanual_accts, officr, on = 'client_id', how = 'left')

rmanual_accts.head()

In [0]:
rmanual_accts[rmanual_accts['client_name'].isna()]

In [0]:
rmanual_accts = rmanual_accts[rmanual_accts['product_id'].isin([24, 26, 29, 33, 212])]
rmanual_accts

In [0]:
with pd.ExcelWriter(f"/Workspace/ReportDump/Tier_Level_Alert/Manually activated {yesterday}.xlsx") as writer:
    rmanual_accts.to_excel(writer, sheet_name = 'Sheet 1', index = False)

In [0]:
# FILE TO SEND AND ITS PATH
filename = f'Manually activated {yesterday}.xlsx'
SourcePathName  = "/Workspace/ReportDump/Tier_Level_Alert/" + filename


import csv, smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders



msg = MIMEMultipart()
msg['From'] = 'auto-report@vfdtech.ng'
msg['To'] = f'compliance@vfd-mfb.com, damilola.fatunmise@vfd-mfb.com, temitope.osinubi@vfd-mfb.com, bright.bassey@vfd-mfb.com, joseph.ogbeche@vfd-mfb.com'
# msg['cc'] = 'productteam@vfd-mfb.com, salesteam@vfd-mfb.com'
msg['Bcc'] = ' data-team@vfdtech.ng'
msg['Subject'] = f'Manual Individual and Corporate accounts activated on {yesterday}'
body = f'Please find attached report for {yesterday}'
msg.attach(MIMEText(body, 'plain'))


## ATTACHMENT PART OF THE CODE IS HERE
attachment = open(SourcePathName, 'rb')
part = MIMEBase('application', "octet-stream")
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(part)

server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.login(username, passwd)  ### if applicable
server.send_message(msg)
server.quit()

## Monthly Manualy opened Accounts report

In [0]:
prev_mth = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m')

curr_mth = datetime.today().strftime('%Y-%m')

if prev_mth == curr_mth:
    print("Current month still running")
else:
    print("Happy new month!\n")
    mth_manual_accts = pd.read_sql(f'''
                select 
                distinct
                dac.client_id,
                dac.client_name,
                dac.client_tier,
                dbw.bvn,
                dab.account_no,
                dbw.restriction_status,
                dbw.narration_description,
                dbw.block_narration_comment,
                daa.product_id,
                daa.product_name,
                dab.balance,
                dab.approvedon_date as activation_date,
                dac.email_address,
                dac.mobile_number,
                dac.mailing_address,
                dab.request_sbumittedby_user_fullname as submission_request_username,
                dab.activatedby_user_fullname
                from dwh_account_balances dab
                left join dwh_all_accounts daa on daa.account_id = dab.account_id
                left join dwh_all_clients dac on dac.client_id = daa.client_id
                left join dwh_bvn_watchlist dbw on dbw.client_id = dac.client_id
                where dab.request_submittedby_userid  != 31
                and left(dab.approvedon_date, 7) = '{prev_mth}'
                ''', conn)
    
    
    ## AML rating score 
    aml = pd.read_sql(f'''
                select
                client_id as client_id,
                final_risk_score as aml_risk_score,
                case when final_risk_score <= 1.5 then 'Low risk'
                    when final_risk_score > 1.5  and final_risk_score < 4 then 'Medium risk'
                    when final_risk_score >= 4 then 'High risk' end as aml_risk_rating
                from aml_customer_risk_rating acr
                where client_id in ({','.join([str(x) for x in mth_manual_accts['client_id'].tolist()])})
                ''', Dumpdb)

    mth_manual_accts = pd.merge(mth_manual_accts, aml, on = 'client_id', how = 'left')
    
    

    officr = pd.read_sql(f'''
                    select
                    a.client_id,
                    dac.client_id as account_officer_id,
                    dac.client_name as account_officer_name
                    from dwh_all_clients dac
                    left join 
                                (select
                                client_id,
                                referred_by_id
                                from dwh_all_clients dcl
                                where client_id in ({','.join([str(x) for x in mth_manual_accts['client_id'].tolist()])})) a on a.referred_by_id = dac.client_id
                    where dac.client_id in (
                                        select
                                        referred_by_id
                                        from dwh_all_clients
                                        where client_id in ({','.join([str(x) for x in mth_manual_accts['client_id'].tolist()])}))
                    ''', conn)

    mth_manual_accts = pd.merge(mth_manual_accts, officr, on = 'client_id', how = 'left')



    mth_manual_accts = mth_manual_accts[mth_manual_accts['product_id'].isin([24, 26, 29, 33, 212])].reset_index(drop = True)
    with pd.ExcelWriter(f"/Workspace/ReportDump/Tier_Level_Alert/Manually activated {prev_mth}.xlsx") as writer:
        mth_manual_accts.to_excel(writer, sheet_name = 'Sheet 1', index = False)
        



    # FILE TO SEND AND ITS PATH
    filename = f'Manually activated {prev_mth}.xlsx'
    SourcePathName  = "/Workspace/ReportDump/Tier_Level_Alert/" + filename


    import csv, smtplib, ssl
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email.mime.text import MIMEText
    from email import encoders



    msg = MIMEMultipart()
    msg['From'] = 'auto-report@vfdtech.ng'
    msg['To'] = f'compliance@vfd-mfb.com, damilola.fatunmise@vfd-mfb.com, temitope.osinubi@vfd-mfb.com, bright.bassey@vfd-mfb.com, joseph.ogbeche@vfd-mfb.com'
    # msg['cc'] = 'productteam@vfd-mfb.com, salesteam@vfd-mfb.com'
    msg['Bcc'] = ' data-team@vfdtech.ng'
    msg['Subject'] = f'Manual Individual and Corporate accounts activated on {prev_mth}'
    body = f'Please find attached report for {prev_mth}'
    msg.attach(MIMEText(body, 'plain'))


    ## ATTACHMENT PART OF THE CODE IS HERE
    attachment = open(SourcePathName, 'rb')
    part = MIMEBase('application', "octet-stream")
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(part)

    server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(username, passwd)  ### if applicable
    server.send_message(msg)
    server.quit()

In [0]:
import gc
gc.collect()

In [0]:
import datetime
last_run_date = datetime.datetime.now()

print("Last run date:", last_run_date)